# AUTA model implementation in pyomo


https://www.pep-net.org/sites/pep-net.org/files/AUTA-en.pdf

## 1 Preparation

### 1.1 Load libraries

In [ ]:
import random
import pyomo.environ as pyo
import pyomo.dataportal.DataPortal as DataPortal
import pandas as pd


### 1.2 Rewrite data

In [ ]:
df = pd.read_excel('data/AUTA.xlsx', 
    sheet_name='AUTA', header=[0], index_col=[0], skiprows=0)
df = df.transpose()
df.to_json('data/AUTA.json')

### 1.3 Helper functions

In [ ]:
## Make some helper functions
sam_dict = {}

def make_init_func(arg, price_level=1, default=None):
    def init_func(*args, price_level=price_level):
        if type(price_level) is dict:
            price_level = price_level[args[1]]
        if (len(arg)):
            if (sam_dict[args[1]][arg] is None):
                return default
            return sam_dict[args[1]][arg]/price_level
        if len(args) == 2:
            if (sam_dict[args[1]] is None):
                return default
            return sam_dict[args[1]]/price_level
        else:
            if (sam_dict[args[1]][args[2]] is None):
                return default
            return sam_dict[args[1]][args[2]]/price_level
    return init_func

def make_sum_sectors(level):
    def init_func(*args):
        if (level == 1):
            return sam_dict[args[1]]['AGR'] + sam_dict[args[1]]['MAN'] + sam_dict[args[1]]['SER']
        elif (level == 2):
            return sam_dict['AGR'][args[1]] + sam_dict['MAN'][args[1]] + sam_dict['SER'][args[1]]
    return init_func

def price_adjust(sam_row, price_level):
    def init_func(*args, price_level=price_level):
        if type(price_level) is dict:
            return sam_row[args[1]]/price_level[args[1]]
        else:
            return sam_row[args[1]]/price_level
    return init_func

### 1.4 Initialize model

In [ ]:
m = pyo.AbstractModel()

## 2 Define sets

In [ ]:
m.sectors = pyo.Set(dimen=1,initialize=['AGR','MAN','SER'], doc='Industries and commodities') 
m.goods = pyo.Set(dimen=1, initialize=['AGR','MAN'], doc='Goods')
m.households = pyo.Set(dimen=1, initialize=['SAL', 'CAP'], doc='Households')

## 3 Load data

In [ ]:
sam_dict = DataPortal()
sam_dict.load(filename='data/AUTA.json', encoding='utf-8')
sam_dict.data()


## 4 Variables and parameters

### 4.1 Variables

In [ ]:
## Nominal variables (values)
m.CTH     = pyo.Var(m.households, doc='Consumption budget of type h households', initialize=lambda _,h: sam_dict['TOT'][h] - sam_dict['ACC'][h])
m.DIV     = pyo.Var(doc='Dividends', initialize=sam_dict['CAP']['F'])
m.IT      = pyo.Var(doc='Total investment', initialize=sam_dict['TOT']['ACC'])
m.SF      = pyo.Var(doc='Business savings', initialize=sam_dict['ACC']['F'])
m.SH      = pyo.Var(m.households, doc='Savings of type h households', initialize=sam_dict['ACC'])
m.YF      = pyo.Var(doc='Business income', initialize=sam_dict['TOT']['F'])
m.YH      = pyo.Var(m.households, doc='Income of type h households', initialize=sam_dict['TOT'])

## Prices (base)
prices = {
    'P': {
        'AGR': 1,
        'MAN': 1,
        'SER': 1
    },
    'R': {
        'AGR': 1,
        'MAN': 1,
        'SER': 1
    },
    'W': 1  
}
m.P       = pyo.Var(m.sectors, doc='Price of commodity i', initialize=prices['P'])
m.R       = pyo.Var(m.sectors, doc='Rental rate of capital in industry j', initialize=prices['R'])
m.W       = pyo.Var(doc='Wage rate', initialize=prices['W'])

# ## Volume variables 
m.C       = pyo.Var(m.sectors, m.households, doc='Consumption of commodity i by type h households', initialize=make_init_func('', prices['P']))
m.CI      = pyo.Var(m.sectors, doc='Total intermediate consumption of industry j', initialize=make_sum_sectors(level=2))
m.DI      = pyo.Var(m.sectors, m.sectors, doc='Intermediate consumption of commodity i in industry j', initialize=make_init_func('',prices['P']))
m.DIT     = pyo.Var(m.sectors, doc='Total intermediate demand of commodity i', initialize=make_sum_sectors(level=1))
m.INV     = pyo.Var(m.sectors, doc='Final demand of commodity i for investment purposes', initialize=make_init_func('ACC', price_level=prices['P'], default=0.0))
m.KD      = pyo.Var(m.sectors, doc='Industry j demand for capital', initialize=price_adjust(sam_dict['KD'], prices['R']))
m.KS      = pyo.Var(m.sectors, doc='Capital supply in industry j', initialize=sam_dict['KD'])
m.LD      = pyo.Var(m.sectors, doc='Industry j demand for labour', initialize=price_adjust(sam_dict['LD'], prices['W']))
m.LS      = pyo.Var(doc='Total labour supply', initialize=sam_dict['LD']['AGR'] + sam_dict['LD']['MAN'] + sam_dict['LD']['SER'])
m.VA      = pyo.Var(m.sectors, doc='Value added of industry j', initialize=lambda _, x : sam_dict['LD'][x] + sam_dict['KD'][x])
m.XS      = pyo.Var(m.sectors, doc='Output of industry j', initialize=price_adjust(sam_dict['TOT'], prices['P']))

## Prices (endogenous)
def init_pci(*args):
     ci_func = make_sum_sectors(level=2)
     return (prices['P']['AGR']*sam_dict['AGR'][args[1]] +
      prices['P']['MAN']*sam_dict['MAN'][args[1]] + 
      prices['P']['SER']*sam_dict['SER'][args[1]]) / ci_func(_, args[1])
m.PCI     = pyo.Var(m.sectors, doc='Intermediate consumption price index of industry j', initialize=init_pci)
def init_pva(*args):
    return (prices['W']*sam_dict['LD'][args[1]] + prices['R'][args[1]]*sam_dict['KD'][args[1]])/(sam_dict['LD'][args[1]] + sam_dict['KD'][args[1]])
m.PVA     = pyo.Var(m.sectors, doc='Price of industry j value added', initialize=init_pva)

## other variables = 
m.LEON    = pyo.Var(doc='Excess supply on the market for services')

### 4.2 Parameters

In [ ]:
def init_alpha(*args):
    
    # WO*LDO(j)/{PVAO(j)*VAO(j)};
    return (prices['W']*sam_dict['LD'][args[1]] / prices['W']) / (init_pva(args[0], args[1]) * (sam_dict['LD'][args[1]] + sam_dict['KD'][args[1]]))

def init_A(*args):
    # VAO(j)/{LDO(j)**alpha(j)*KDO(j)**(1-alpha(j))};
    alpha = init_alpha(args[0], args[1])
    va = sam_dict['LD'][args[1]] + sam_dict['KD'][args[1]]
    ld = price_adjust(sam_dict['LD'], prices['W'])(args[0], args[1])
    kd = price_adjust(sam_dict['KD'], prices['R'])(args[0], args[1])
    return va/(ld**alpha * kd**(1-alpha))

def init_v(*args): 
    va = sam_dict['LD'][args[1]] + sam_dict['KD'][args[1]]
    return va/(sam_dict['TOT'][args[1]] / prices['P'][args[1]])

def init_io(*args):
    ci_func = make_sum_sectors(level=2)
    return ci_func(args[0], args[1]) / (sam_dict['TOT'][args[1]] / prices['P'][args[1]])

def init_aij(*args):
    ci_func = make_sum_sectors(level=2)
    di_func = make_init_func('', prices['P'])
    return di_func(args[0], args[1], args[2]) / ci_func(args[0], args[2])

def init_gamma(*args):
    # PO(i)*CO(i,h)/CTHO(h);
    # no price adjustment because function expects price-adjusted inputs
    cth= sam_dict['TOT'][args[2]] - sam_dict['ACC'][args[2]]
    return sam_dict[args[1]][args[2]]/ cth

def init_lmbda(*args):
    #{YHO('cap')-DIVO}/SUM[j,RO(j)*KDO(j)];
    return (sam_dict['TOT']['CAP'] - sam_dict['CAP']['F']) / (sam_dict['KD']['AGR'] + sam_dict['KD']['MAN'] + sam_dict['KD']['SER'])

def init_mu(*args):
    # PO(i)*INVO(i)/ITO;
    inv = make_init_func('ACC', price_level=prices['P'], default=0.0)
    return prices['P'][args[1]]*inv(args[0], args[1]) / sam_dict['TOT']['ACC']

def init_psi(*args):
    #SHO(h)/YHO(h);
    return sam_dict['ACC'][args[1]] / sam_dict['TOT'][args[1]]

m.A         = pyo.Param(m.sectors, doc='Scale parameter (Cobb-Douglas - Production function)', initialize=init_A)
m.aij       = pyo.Param(m.sectors, m.sectors, doc='Scale parameter (Cobb-Douglas - Production function)', initialize=init_aij)
m.alpha     = pyo.Param(m.sectors, doc='Elasticity (Cobb-Douglas production function)', initialize=init_alpha)
m.gamma     = pyo.Param(m.sectors, m.households, doc='Share of commodity i in type h household consumption budget', initialize=init_gamma)
m.io        = pyo.Param(m.sectors, doc='Coefficient (Leontief - total intermediate consumption)', initialize=init_io)
m.lmbda     = pyo.Param(doc='Share of capital income receied by capitalists', initialize=init_lmbda)
m.mu        = pyo.Param(m.sectors, doc='Share of commodity i in total investment expenditures', initialize=init_mu)
m.psi       = pyo.Param(m.households, doc='Average propensity to save of type h household', initialize=init_psi)
m.v         = pyo.Param(m.sectors, doc='Coefficient (Leontief - value added)', initialize=init_v)

## 5 Equations

In [ ]:
## Production
def XSEQ(m, j):
    return m.VA[j] == m.v[j]*m.XS[j]
m.XSEQ = pyo.Constraint(m.sectors, rule=XSEQ, doc='Value added demand in industry j (Leontief)')

def CIEQ(m, j):
    return m.CI[j] == m.io[j]*m.XS[j]
m.CIEQ = pyo.Constraint(m.sectors, rule=CIEQ, doc='Total intermediate consumption demand in industry j (Leontief)')

def VAEQ(m, j):
    return m.VA[j] == m.A[j]*(m.LD[j]**m.alpha[j])*(m.KD[j]**(1-m.alpha[j]))
m.VAEQ = pyo.Constraint(m.sectors, rule=VAEQ, doc='Cobb-Douglas between labour and capital')

def LDEQ(m, j):
    return m.W*m.LD[j] == m.alpha[j]*m.PVA[j]*m.VA[j]
m.LDEQ = pyo.Constraint(m.sectors, rule=LDEQ, doc='Demand for labour by industry j')

def KDEQ(m, j):
    return m.R[j]*m.KD[j] == (1-m.alpha[j])*m.PVA[j]*m.VA[j]
m.KDEQ = pyo.Constraint(m.sectors, rule=KDEQ, doc='Demand for capital by industry j')

def DIEQ(m, i, j):
    return m.DI[(i, j)] == m.aij[(i,j)]*m.CI[j]
m.DIEQ = pyo.Constraint(m.sectors, m.sectors, rule=DIEQ, doc='Intermediate consumption of commodity i by sector j')


In [ ]:
## Income and savings
def YHSEQ(m):
    return m.YH['SAL'] == m.W*sum(m.LD[j] for j in m.sectors)
m.YHSEQ = pyo.Constraint(rule=YHSEQ, doc='Household income (workers)')

def YHCEQ(m):
    return m.YH['CAP'] == m.lmbda*sum(m.R[j]*m.KD[j] for j in m.sectors) + m.DIV
m.YHCEQ = pyo.Constraint(rule=YHCEQ, doc='Household income (workers)')

def SHEQ(m, h):
    return m.SH[h] == m.psi[h]*m.YH[h]
m.SHEQ = pyo.Constraint(m.households, rule=SHEQ, doc='Household h savings')

def CTHEQ(m, h):
    return m.CTH[h] == m.YH[h] - m.SH[h]
m.CTHEQ = pyo.Constraint(m.households, rule=CTHEQ, doc='Consumption budget for household h')

def YFEQ(m):
    return m.YF == (1-m.lmbda)*sum(m.R[j]*m.KD[j] for j in m.sectors) 
m.YFEQ = pyo.Constraint(rule=YFEQ, doc='Firms income')

def SFEQ(m):
    return m.SF == m.YF - m.DIV
m.SFEQ = pyo.Constraint(rule=SFEQ, doc='Firms savings')
 

In [ ]:
## Demand
def CEQ(m, i, h):
    return m.P[i]*m.C[(i,h)] == m.gamma[(i,h)]*m.CTH[h]
m.CEQ = pyo.Constraint(m.sectors, m.households, rule=CEQ, doc='Household h consumption of commodity i')

def INVEQ(m, j):
    return m.P[j]*m.INV[j] == m.mu[j]*m.IT
m.INVEQ = pyo.Constraint(m.sectors, rule=INVEQ, doc='Investment in commodity i')

def DITEQ(m, i):
    return m.DIT[i] == sum(m.DI[(i,j)] for j in m.sectors)
m.DITEQ = pyo.Constraint(m.sectors, rule=DITEQ, doc='Intermediate demand for commodity i')

In [ ]:
## Prices
def PCIEQ(m, j):
    return m.PCI[j]*m.CI[j] == sum(m.P[i]*m.DI[(i,j)] for i in m.sectors)
m.PCIEQ = pyo.Constraint(m.sectors, rule=PCIEQ, doc='Intermediate consumption price index')

def CPEQ(m, j):
    return m.P[j]*m.XS[j] == m.PVA[j]*m.VA[j] + m.PCI[j]*m.CI[j]
m.CPEQ = pyo.Constraint(m.sectors, rule=CPEQ, doc='Production costs for sector j')



In [ ]:
## Equilibrium
def PEQ(m, bns):
    return m.XS[bns] == sum(m.C[(bns, h)] for h in m.households) + m.DIT[bns] + m.INV[bns]
m.PEQ = pyo.Constraint(m.goods, rule=PEQ, doc='Domestic absorption (over goods)')

def WEQ(m):
    return m.LS == sum(m.LD[j] for j in m.sectors)
m.WEQ = pyo.Constraint(rule=WEQ, doc='Labour market equilibrium')

def REQ(m, j):
    return m.KS[j] == m.KD[j]
m.REQ = pyo.Constraint(m.sectors, rule=REQ, doc='Capital market equilibrium')

def ITEQ(m):
    return m.IT == sum(m.SH[h] for h in m.households) + m.SF
m.ITEQ = pyo.Constraint(rule=ITEQ, doc='Investment-savings equilibrium')

## Other
def WALRAS(m):
    return m.LEON == m.XS['SER'] - sum(m.C[('SER', h)] for h in m.households) - m.DIT['SER'] - m.INV['SER']
m.WALRAS =  pyo.Constraint(rule=WALRAS, doc="Verification of Walras' law")


## 6 Create instances

### 6.1 Helper functions

In [ ]:
def count_nonfixed(i, active=True):
    num_non_fixed = 0
    for block in i.block_data_objects(active=active):
        for data in block.component_map(pyo.Var, active=active).values():
            for key in data.keys():
                if not data[key].is_fixed():
                    num_non_fixed += 1
    return num_non_fixed

In [ ]:
def applyFixes(inst):
    inst.P['AGR'].fix() #1 numeraire
    inst.KS.fix()       #3
    inst.LS.fix()       #1
    inst.DIV.fix()      #1

### 6.2 Define optimization objective

In [ ]:
def obj_expression(m):
    return sum(m.C[(j, h)] for j, h in m.sectors*m.households)

m.OBJ = pyo.Objective(rule=obj_expression, sense=pyo.maximize)

### 6.3 Create base instance

In [ ]:
inst_base = m.create_instance()
applyFixes(inst_base)

# Check for consistency
print('constraints:', inst.nconstraints())
print('variables:', inst.nvariables())
print('unfixed', count_nonfixed(inst))
if (count_nonfixed(inst) == inst.nconstraints()):
    print('Model is square.')
else:
    print('Model is NOT SQUARE!')

In [ ]:
# Display some of the parameters
inst_base.A.display()
inst_base.alpha.display()
inst_base.io.display()
inst_base.v.display()
inst_base.aij.display()
inst_base.gamma.display()
inst_base.psi.display()
inst_base.mu.display()
inst_base.lmbda.display()

### 6.4 Create scenarios

In [ ]:
# Capital shock on service sector
inst_scenario_1 = m.create_instance()
applyFixes(inst_scenario_1)
inst_scenario_1.KS['SER'] = 1.1 * inst_scenario_1.KS['SER'].value

# Shock to numeraire
inst_scenario_2 = m.create_instance()
applyFixes(inst_scenario_2)
inst_scenario_2.P['AGR'] = 1.5 * inst_scenario_2.P['AGR'].value
inst_scenario_2.DIV = 1.5 * inst_scenario_2.DIV.value

## 7 Solve instances
This should be fast as the model is already at equilibrium.

In [ ]:
opt = pyo.SolverFactory('ipopt')
opt.solve(inst_base)

In [ ]:
opt.solve(inst_scenario_1)

In [ ]:
opt.solve(inst_scenario_2)

## 8 Display results

In [ ]:
inst_base.display()

In [ ]:
inst_scenario_1.display()

In [ ]:
inst_scenario_2.display()